<a href="https://colab.research.google.com/github/aashrithpy/cassandra/blob/main/BigData_cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install cassandra-driver


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 34.3 MB/s eta 0:00:00


In [5]:
pip install --upgrade astrapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.4 MB/s eta 0:00:00


In [11]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json


In [13]:
cloud_config= {
        'secure_connect_bundle': 'secure-connect-bigdata.zip'
}
with open("bigdata-token.json") as f:
    secrets = json.load(f)
CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]
auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
if session:
    print("Session is active")
else:
    print("Session is not active")

Session is active


In [14]:
create_table_query = """
CREATE TABLE sales.sales_data (
    Region TEXT,
    Country TEXT,
    Item_Type TEXT,
    Sales_Channel TEXT,
    Order_Priority TEXT,
    Order_Date TEXT,
    Order_ID BIGINT PRIMARY KEY,
    Ship_Date TEXT,
    Units_Sold INT,
    Unit_Price FLOAT,
    Unit_Cost FLOAT,
    Total_Revenue FLOAT,
    Total_Cost FLOAT,
    Total_Profit FLOAT
);
"""
session.execute(create_table_query)
print("Table created successfully!")

Table created successfully!


In [16]:
import pandas as pd
from cassandra.cluster import Cluster

csv_file_path = "sales_100.csv"
df = pd.read_csv(csv_file_path)

for _, row in df.iterrows():
    query = """
    INSERT INTO sales.sales_data (Region, Country, Item_Type, Sales_Channel, Order_Priority, Order_Date, Order_ID, Ship_Date, Units_Sold, Unit_Price, Unit_Cost, Total_Revenue, Total_Cost, Total_Profit)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    session.execute(query, tuple(row))

print("Data imported successfully!")


Data imported successfully!


In [19]:
query = "SELECT * FROM sales.sales_data ;"
rows = session.execute(query)

for row in rows:
    print(row)

Row(order_id=294530856, country='Italy', item_type='Cereal', order_date='11/15/2011', order_priority='M', region='Europe', sales_channel='Online', ship_date='12/28/2011', total_cost=829138.8125, total_profit=627217.1875, total_revenue=1456356.0, unit_cost=117.11000061035156, unit_price=205.6999969482422, units_sold=7080)
Row(order_id=274930989, country='Dominica', item_type='Household', order_date='11/19/2011', order_priority='C', region='Central America and the Caribbean', sales_channel='Offline', ship_date='12/13/2011', total_cost=3539891.75, total_profit=1167402.125, total_revenue=4707294.0, unit_cost=502.5400085449219, unit_price=668.27001953125, units_sold=7044)
Row(order_id=498071897, country='Taiwan', item_type='Cereal', order_date='4/11/2010', order_priority='H', region='Asia', sales_channel='Online', ship_date='5/26/2010', total_cost=1100482.625, total_profit=832480.25, total_revenue=1932962.875, unit_cost=117.11000061035156, unit_price=205.6999969482422, units_sold=9397)
Row(

In [75]:

create_silver_table_query = """
CREATE TABLE IF NOT EXISTS sales.silver_sales_data (
    order_id UUID PRIMARY KEY,
    region TEXT,
    country TEXT,
    item_type TEXT,
    sales_channel TEXT,
    order_priority TEXT,
    order_date DATE,
    ship_date DATE,
    units_sold INT,
    unit_price DECIMAL,
    unit_cost DECIMAL,
    total_revenue DECIMAL,
    total_cost DECIMAL,
    total_profit DECIMAL
);


"""

session.execute(create_silver_table_query)



In [79]:
import pandas as pd
from cassandra.cluster import Cluster
from datetime import datetime

csv_file_path = "sales_100.csv"
df = pd.read_csv(csv_file_path)

df.columns = df.columns.str.strip()

print("Column names in the DataFrame:", df.columns)

df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%m/%d/%Y')

required_columns = ['Order ID', 'Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority',
                    'Order Date', 'Ship Date', 'UnitsSold', 'UnitPrice', 'UnitCost', 'TotalRevenue',
                    'TotalCost', 'TotalProfit']

missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"Missing columns in the DataFrame: {missing_columns}")
else:

    for _, row in df.iterrows():

        query = """
        INSERT INTO sales.silver_sales_data (
            order_id, region, country, item_type, sales_channel, order_priority,
            order_date, ship_date, units_sold, unit_price, unit_cost, total_revenue,
            total_cost, total_profit
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        session.execute(query, (
            row['Order ID'],  # UUID type
            row['Region'],  # TEXT type
            row['Country'],  # TEXT type
            row['Item Type'],  # TEXT type
            row['Sales Channel'],  # TEXT type
            row['Order Priority'],  # TEXT type
            row['Order Date'].date(),  # Convert to DATE format
            row['Ship Date'].date(),  # Convert to DATE format
            int(row['UnitsSold']),  # INT type
            float(row['UnitPrice']),  # DECIMAL type
            float(row['UnitCost']),  # DECIMAL type
            float(row['TotalRevenue']),  # DECIMAL type
            float(row['TotalCost']),  # DECIMAL type
            float(row['TotalProfit'])  # DECIMAL type
        ))

    print("Silver table populated successfully!")


Column names in the DataFrame: Index(['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority',
       'Order Date', 'Order ID', 'Ship Date', 'UnitsSold', 'UnitPrice',
       'UnitCost', 'TotalRevenue', 'TotalCost', 'TotalProfit'],
      dtype='object')
Silver table populated successfully!


In [81]:
query = "SELECT * FROM sales.silver_sales_data ;"
rows = session.execute(query)

for row in rows:
    print(row)

Row(order_id=294530856, country='Italy', item_type='Cereal', order_age=None, order_date='2011-11-15', order_priority='M', profit_per_unit=None, region='Europe', revenue_per_unit=None, sales_channel='Online', ship_date='2011-12-28', total_cost=829138.8125, total_profit=627217.1875, total_revenue=1456356.0, unit_cost=117.11000061035156, unit_price=205.6999969482422, units_sold=7080)
Row(order_id=274930989, country='Dominica', item_type='Household', order_age=None, order_date='2011-11-19', order_priority='C', profit_per_unit=None, region='Central America and the Caribbean', revenue_per_unit=None, sales_channel='Offline', ship_date='2011-12-13', total_cost=3539891.75, total_profit=1167402.125, total_revenue=4707294.0, unit_cost=502.5400085449219, unit_price=668.27001953125, units_sold=7044)
Row(order_id=498071897, country='Taiwan', item_type='Cereal', order_age=None, order_date='2010-04-11', order_priority='H', profit_per_unit=None, region='Asia', revenue_per_unit=None, sales_channel='Onli

In [65]:
#Total_profit, Total_sales,Total_units_sold by country and Item_type
from cassandra.cluster import Cluster
from datetime import datetime

create_gold_table_query = """
CREATE TABLE IF NOT EXISTS sales.gold_table (
    country TEXT,
    item_type TEXT,
    total_sales TEXT,
    total_profit TEXT,
    average_profit_margin TEXT,
    total_units_sold TEXT,
    PRIMARY KEY (country, item_type)
);
"""
session.execute(create_gold_table_query)

select_query = """
SELECT country, item_type, total_revenue, total_profit, units_sold
FROM sales.sales_data;
"""

rows = session.execute(select_query)

insert_query = """
INSERT INTO sales.gold_table (country, item_type, total_sales, total_profit, average_profit_margin, total_units_sold)
VALUES (%s, %s, %s, %s, %s, %s)
"""

aggregated_data = {}

for row in rows:
    key = (row.country, row.item_type)
    if key not in aggregated_data:
        aggregated_data[key] = {
            'total_revenue': 0,
            'total_profit': 0,
            'total_units_sold': 0
        }

    aggregated_data[key]['total_revenue'] += row.total_revenue
    aggregated_data[key]['total_profit'] += row.total_profit
    aggregated_data[key]['total_units_sold'] += row.units_sold

for key, values in aggregated_data.items():
    country, item_type = key
    total_revenue = str(values['total_revenue'])
    total_profit = str(values['total_profit'])
    total_units_sold = str(values['total_units_sold'])
    average_profit_margin = str(values['total_profit'] / values['total_revenue']) if values['total_revenue'] != 0 else '0'

    session.execute(insert_query, (
        str(country), str(item_type), total_revenue, total_profit, average_profit_margin, total_units_sold
    ))

print("Gold table populated successfully!")


Gold table populated successfully!


In [66]:
query = "SELECT * FROM sales.gold_table ;"
rows = session.execute(query)

for row in rows:
    print(row)

Row(country='Malaysia', item_type='Beverages', average_profit_margin='0.33003160416460126', total_profit='143351.640625', total_sales='434357.3125', total_units_sold='9154')
Row(country='Israel', item_type='Beverages', average_profit_margin='0.33003160565054235', total_profit='73742.9375', total_sales='223442.046875', total_units_sold='4709')
Row(country='Serbia', item_type='Clothes', average_profit_margin='0.6720351474112635', total_profit='539637.125', total_sales='802989.4375', total_units_sold='7348')
Row(country='Djibouti', item_type='Clothes', average_profit_margin='0.6720351662844927', total_profit='41273.28125', total_sales='61415.359375', total_units_sold='562')
Row(country='Egypt', item_type='Clothes', average_profit_margin='0.6720351239886274', total_profit='87540.4765625', total_sales='130261.7578125', total_units_sold='1192')
Row(country='Nicaragua', item_type='Baby Food', average_profit_margin='0.3755092210247902', total_profit='277131.25', total_sales='738014.5', total_u

In [85]:
# Total_Sales by Item_type
create_gold_table_query_2 = """
CREATE TABLE IF NOT EXISTS sales.gold_sales_by_item_type (
    item_type TEXT PRIMARY KEY,
    total_revenue DECIMAL,
    total_profit DECIMAL,
    average_profit_margin DECIMAL,
    total_units_sold INT
)
"""
session.execute(create_gold_table_query_2)

select_query = """
SELECT item_type, total_revenue, total_profit, units_sold FROM sales.silver_sales_data
"""

rows = session.execute(select_query)

aggregated_data = {}

for row in rows:
    item_type = row.item_type
    total_revenue = row.total_revenue
    total_profit = row.total_profit
    units_sold = row.units_sold

    if item_type not in aggregated_data:
        aggregated_data[item_type] = {
            'total_revenue': 0,
            'total_profit': 0,
            'total_units_sold': 0
        }

    aggregated_data[item_type]['total_revenue'] += total_revenue
    aggregated_data[item_type]['total_profit'] += total_profit
    aggregated_data[item_type]['total_units_sold'] += units_sold

for item_type, data in aggregated_data.items():
    total_revenue = data['total_revenue']
    total_profit = data['total_profit']
    total_units_sold = data['total_units_sold']

    average_profit_margin = total_profit / total_revenue if total_revenue != 0 else 0

    insert_query = """
    INSERT INTO sales.gold_sales_by_item_type (
        item_type, total_revenue, total_profit, average_profit_margin, total_units_sold
    )
    VALUES (%s, %s, %s, %s, %s)
    """
    session.execute(insert_query, (
        item_type,
        float(total_revenue),
        float(total_profit),
        float(average_profit_margin),
        int(total_units_sold)
    ))

print("Gold table 2 (sales by item type) populated successfully!")


Gold table 2 (sales by item type) populated successfully!


In [86]:
query = "SELECT * FROM sales.gold_sales_by_item_type ;"
rows = session.execute(query)

for row in rows:
    print(row)

Row(item_type='Household', average_profit_margin=Decimal('0.24799856264263057'), total_profit=Decimal('9552677.1640625'), total_revenue=Decimal('38519082.78125'), total_units_sold=57640)
Row(item_type='Office Supplies', average_profit_margin=Decimal('0.19386987389220692'), total_profit=Decimal('5405267.5'), total_revenue=Decimal('27880904.8125'), total_units_sold=42814)
Row(item_type='Vegetables', average_profit_margin=Decimal('0.4097754086540353'), total_profit=Decimal('465141.83203125'), total_revenue=Decimal('1135114.0703125'), total_units_sold=7368)
Row(item_type='Snacks', average_profit_margin=Decimal('0.3613841827905915'), total_profit=Decimal('792747.78125'), total_revenue=Decimal('2193642.71875'), total_units_sold=14377)
Row(item_type='Personal Care', average_profit_margin=Decimal('0.30661935895585124'), total_profit=Decimal('978467.7001953125'), total_revenue=Decimal('3191147.82421875'), total_units_sold=39045)
Row(item_type='Meat', average_profit_margin=Decimal('0.13558036346

In [87]:
#Sales by region and country
create_gold_table_query_3 = """
CREATE TABLE IF NOT EXISTS sales.gold_sales_by_region_or_country (
    region TEXT,
    country TEXT,
    total_revenue DECIMAL,
    total_profit DECIMAL,
    total_units_sold INT,
    PRIMARY KEY (region, country)
)
"""

session.execute(create_gold_table_query_3)

select_query = """
SELECT region, country, total_revenue, total_profit, units_sold FROM sales.silver_sales_data
"""

rows = session.execute(select_query)

aggregated_data_by_region_country = {}

for row in rows:
    region = row.region
    country = row.country
    total_revenue = row.total_revenue
    total_profit = row.total_profit
    units_sold = row.units_sold

    if (region, country) not in aggregated_data_by_region_country:
        aggregated_data_by_region_country[(region, country)] = {
            'total_revenue': 0,
            'total_profit': 0,
            'total_units_sold': 0
        }

    aggregated_data_by_region_country[(region, country)]['total_revenue'] += total_revenue
    aggregated_data_by_region_country[(region, country)]['total_profit'] += total_profit
    aggregated_data_by_region_country[(region, country)]['total_units_sold'] += units_sold

sorted_data = sorted(
    aggregated_data_by_region_country.items(),
    key=lambda x: x[1]['total_profit'],
    reverse=True
)

for (region, country), data in sorted_data:
    total_revenue = data['total_revenue']
    total_profit = data['total_profit']
    total_units_sold = data['total_units_sold']

    insert_query = """
    INSERT INTO sales.gold_sales_by_region_or_country (
        region, country, total_revenue, total_profit, total_units_sold
    )
    VALUES (%s, %s, %s, %s, %s)
    """
    session.execute(insert_query, (
        region,
        country,
        float(total_revenue),
        float(total_profit),
        int(total_units_sold)
    ))

print("Gold table 3 (sales by region/country) populated successfully!")


Gold table 3 (sales by region/country) populated successfully!


In [88]:
query = "SELECT * FROM sales.gold_sales_by_region_or_country ;"
rows = session.execute(query)

for row in rows:
    print(row)

Row(region='Australia and Oceania', country='East Timor', total_profit=Decimal('22944.810546875'), total_revenue=Decimal('53276.30078125'), total_units_sold=259)
Row(region='Australia and Oceania', country='New Zealand', total_profit=Decimal('90640.078125'), total_revenue=Decimal('274640.59375'), total_units_sold=5788)
Row(region='Australia and Oceania', country='Papua New Guinea', total_profit=Decimal('688308.5'), total_revenue=Decimal('1145454.15625'), total_units_sold=9452)
Row(region='Australia and Oceania', country='Samoa ', total_profit=Decimal('937534.625'), total_revenue=Decimal('3780403.5'), total_units_sold=5657)
Row(region='Australia and Oceania', country='Solomon Islands', total_profit=Decimal('700209.25'), total_revenue=Decimal('2823440.75'), total_units_sold=4225)
Row(region='Australia and Oceania', country='Tonga', total_profit=Decimal('530201.6875'), total_revenue=Decimal('1411953.625'), total_units_sold=5531)
Row(region='Australia and Oceania', country='Vanuatu', total

In [90]:
# Average shipping time by region
create_gold_table_query_4 = """
CREATE TABLE IF NOT EXISTS sales.gold_average_shipping_time_by_region (
    region TEXT PRIMARY KEY,
    average_shipping_time DECIMAL
)
"""

session.execute(create_gold_table_query_4)

select_query = """
SELECT region, order_date, ship_date FROM sales.silver_sales_data
"""

rows = session.execute(select_query)

shipping_time_data = {}
for row in rows:
    region = row.region
    order_date = datetime.strptime(row.order_date, "%Y-%m-%d")
    ship_date = datetime.strptime(row.ship_date, "%Y-%m-%d")
    shipping_time = (ship_date - order_date).days

    if region not in shipping_time_data:
        shipping_time_data[region] = {
            'total_shipping_time': 0,
            'order_count': 0
        }

    shipping_time_data[region]['total_shipping_time'] += shipping_time
    shipping_time_data[region]['order_count'] += 1

average_shipping_time_by_region = {
    region: data['total_shipping_time'] / data['order_count']
    for region, data in shipping_time_data.items()
}
for region, avg_shipping_time in average_shipping_time_by_region.items():
    insert_query = """
    INSERT INTO sales.gold_average_shipping_time_by_region (region, average_shipping_time)
    VALUES (%s, %s)
    """
    session.execute(insert_query, (region, float(avg_shipping_time)))

print("Gold table 4 (average shipping time by region) populated successfully!")


Gold table 4 (average shipping time by region) populated successfully!


In [91]:
query = "SELECT * FROM sales.gold_average_shipping_time_by_region ;"
rows = session.execute(query)

for row in rows:
    print(row)

Row(region='Australia and Oceania', average_shipping_time=Decimal('19.88888888888889'))
Row(region='Europe', average_shipping_time=Decimal('24.625'))
Row(region='Middle East and North Africa', average_shipping_time=Decimal('28.0'))
Row(region='Central America and the Caribbean', average_shipping_time=Decimal('26.636363636363637'))
Row(region='Asia', average_shipping_time=Decimal('29.94736842105263'))
Row(region='Sub-Saharan Africa', average_shipping_time=Decimal('19.458333333333332'))
Row(region='North America', average_shipping_time=Decimal('26.5'))
